In [1]:
from pyspark.sql import SparkSession

# إنشاء SparkSession
spark = SparkSession.builder \
    .appName("NullHandlingExample") \
    .getOrCreate()
from pyspark.sql import SparkSession

# إنشاء SparkSession
spark = SparkSession.builder \
    .appName("NullHandlingExample") \
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/29 00:30:16 WARN Utils: Your hostname, ahmed-refat-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/10/29 00:30:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/29 00:30:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# هنا بنريد  الاستريم من الجهاز عن طريق السوكيت  
# ونقدر نعمل كده لو كتبا في التريمينل كده
# nc  -l 1412                 -----1412  are port  numberr

df=spark.readStream.format('socket')\
    .option('host','localhost')\
        .option('port',1412).load()
df

25/10/29 00:32:02 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


DataFrame[value: string]

In [ ]:
# هنا  بنعمل رايت ل الدااتا وبنعمل تريجر ل عشان يروح يششوف الداتاكل 5 ثوواني 
writer = df.writeStream.outputMode("append").format("console").trigger(processingTime="5 seconds")
#query.awaitTermination()
query= writer.start()

25/10/29 00:26:04 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
25/10/29 00:26:04 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-028c24ff-3004-4840-888a-109e3abcdc14. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/29 00:26:04 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [21]:
# لو هنوقف الاستريم 
#query.stop()

هنعمل مثال كامل 
---------------
====================================================================================================

25/10/29 00:44:56 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
